In [27]:
import numpy as np
import pandas as pd
import re
import datetime
import warnings
import sys

from miscfns.misc_fns import db_con,describe_dupe_cor_ans, get_item_cor_ans, recode_as_omitted, timing_exclusion, remove_repeat_questions, combine_CIinfo, removed_record_count, clean_item_data
from miscfns.misc_fns import make_user_level_matrices, make_item_level_info, make_activity_level_info, make_user_level_info

from miscfns.misc_fns import color

# Reading the data

## COMLEX

In [28]:
program = 'COMLEX'
analysis_name = program
results_path = 'C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\MED\\Results\\'+program+'\\'
path = 'C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\MED\\Data\\'+program+'\\'

In [29]:
activity_df = pd.read_csv(path + 'activity_info.tsv', sep = '\t')

response_df = pd.read_csv(path + 'response_data.tsv', sep = '\t')
response_df['content_item_name'] = response_df['content_item_name'].str.lower()

content_df = pd.read_csv(path + 'content_info.tsv', sep = '\t')
content_df['content_item_name'] = content_df['content_item_name'].str.lower()

In [4]:
#reading items file
comlex_items = pd.read_excel('C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\MED\\COMLEX and Step 2 Qbank Item lists.xlsx', sheet_name= 'COMLEX Level 1')
comlex_items.rename(columns = {'Parent QID': 'content_item_name'}, inplace=True)
comlex_items['content_item_name'] = comlex_items['content_item_name'].str.lower()


In [5]:
#common items summary
common_items_comlex = pd.merge(content_df[['content_item_name', 'content_item_id']], comlex_items['content_item_name'], left_on=['content_item_name'], right_on=['content_item_name']).shape[0]

print('Common items out of', color.BOLD + str(comlex_items['content_item_name'].nunique()) + color.END , 'for COMLEX :', color.BOLD + str(common_items_comlex) + color.END)



Common items out of 1134 for COMLEX : 884


In [ ]:
response_df = pd.merge(response_df, comlex_items['content_item_name'], on=['content_item_name'])
activity_df = pd.merge(activity_df, response_df['activity_id'].drop_duplicates(), on = ['activity_id'])
content_df = pd.merge(content_df, comlex_items['content_item_name'], on=['content_item_name'])


In [ ]:
print('Shape of activity_df :', activity_df.shape)
print('Total Students : ', activity_df['student_id'].nunique())
print('Total Activities : ', activity_df['activity_id'].nunique())
print('Total Enrollments : ', activity_df['enrollment_id'].nunique())
print('Template Id : ', activity_df['template_id'].unique())

print('-'*50)

print('Shape of response_df :', response_df.shape)
print('Total Students : ', response_df['student_id'].nunique())
print('Total Activities : ', response_df['activity_id'].nunique())
print('Total Content Items : ', response_df['content_item_id'].nunique())

print('-'*50)

print('Shape of content_df :', content_df.shape)
print('Total Content Items : ', content_df['content_item_id'].nunique())
print('Is Correct_answer column available? ', 'correct_answer' in content_df.columns)
print('Data available in correct answer columns :', content_df['correct_answer'].nunique())

## USMLE

In [32]:
program = 'USMLE'
analysis_name = program
results_path = 'C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\MED\\Results\\'+program+'\\'
path = 'C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\MED\\Data\\'+program+'\\'

In [33]:
activity_df = pd.read_csv(path + 'activity_info.tsv', sep = '\t')

response_df = pd.read_csv(path + 'response_data.tsv', sep = '\t')
response_df['content_item_name'] = response_df['content_item_name'].str.lower()

content_df = pd.read_csv(path + 'content_info.tsv', sep = '\t')
content_df['content_item_name'] = content_df['content_item_name'].str.lower()

In [34]:
usmle_items = pd.read_excel('C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\MED\\COMLEX and Step 2 Qbank Item lists.xlsx', sheet_name= 'USMLE Step 2 items')
usmle_items.rename(columns = {'QID': 'content_item_name'}, inplace=True)
usmle_items['content_item_name'] = usmle_items['content_item_name'].str.lower()

In [35]:
#common items summary
common_items_usmle = pd.merge(content_df[['content_item_name', 'content_item_id']], usmle_items['content_item_name'], left_on=['content_item_name'], right_on=['content_item_name']).shape[0]

print('Common items out of', color.BOLD + str(usmle_items['content_item_name'].nunique()) + color.END , 'for USMLE :', color.BOLD + str(common_items_usmle) + color.END)

Common items out of 1135 for USMLE : 444


In [36]:
response_df = pd.merge(response_df, usmle_items['content_item_name'], on=['content_item_name'])
activity_df = pd.merge(activity_df, response_df['activity_id'].drop_duplicates(), on = ['activity_id'])
content_df = pd.merge(content_df, usmle_items['content_item_name'], on=['content_item_name'])

In [40]:
print('Shape of activity_df :', activity_df.shape)
print('Total Students : ', activity_df['student_id'].nunique())
print('Total Activities : ', activity_df['activity_id'].nunique())
print('Total Enrollments : ', activity_df['enrollment_id'].nunique())
print('Template Id : ', activity_df['template_id'].unique())

print('-'*50)

print('Shape of response_df :', response_df.shape)
print('Total Students : ', response_df['student_id'].nunique())
print('Total Activities : ', response_df['activity_id'].nunique())
print('Total Content Items : ', response_df['content_item_id'].nunique())

print('-'*50)

print('Shape of content_df :', content_df.shape)
print('Total Content Items : ', content_df['content_item_id'].nunique())
print('Is Correct_answer column available? ', 'correct_answer' in content_df.columns)
print('Data available in correct answer columns :', content_df['correct_answer'].nunique())

Shape of activity_df : (47948, 35)
Total Students :  3203
Total Activities :  47948
Total Enrollments :  3479
Template Id :  [12879]
--------------------------------------------------
Shape of response_df : (263674, 20)
Total Students :  3203
Total Activities :  47948
Total Content Items :  444
--------------------------------------------------
Shape of content_df : (444, 35)
Total Content Items :  444
Is Correct_answer column available?  True
Data available in correct answer columns : 0


# Data handling

In [41]:
response_df.rename(columns={'history_db_id': 'historiesDb',
                            'student_id' : 'jasperUserId',
                            'activity_id' : 'sequenceId',
                            'item_position': 'position',
                            'section_title' : 'sectionTitle',
                            'item_section_position' : 'displaySeq',
                            'content_item_id' : 'contentItemId',
                            'content_item_name' : 'contentItemName',
                            'interaction_type': 'interactionType',
                            'milliseconds_used' : 'mSecUsed',
                            'is_scored' : 'scored',
                            'scored_response' : 'score',
                            'raw_response' : 'response',
                            'field_test' : 'fieldTest',
                           'item_status': 'responseStatus'}, inplace = True)
#response_df['correctAnswer'] = np.nan
response_df['sectionName'] = response_df['sectionTitle']

#removing responses with null display sequence
#print('Before removing null displaySeq : ', response_df.shape)

#response_df = response_df[~response_df['displaySeq'].isnull()]

#print('After removing null displaySeq : ', response_df.shape)

In [42]:
activity_df.rename(columns={'history_db_id': 'historiesDb',
                            'student_id' : 'jasperUserId',
                            'enrollment_id' : 'kbsEnrollmentId',
                            'activity_id' : 'sequenceId',
                            'template_id' : 'templateId',
                            'template_name' : 'sequenceName',
                            'sequence_title' : 'sequenceTitle',
                            'timestamp_created' : 'dateCreated',
                            'timestamp_completed' : 'dateCompleted',
                           'tutor_mode' : 'tutorMode',
                           'status':'sequenceStatus'}, inplace = True)

activity_df['dateCreated'] = pd.to_datetime(activity_df['dateCreated'], errors = 'coerce')
activity_df['dateCompleted'] = pd.to_datetime(activity_df['dateCompleted'], errors = 'coerce')


#merging responses and activity data and changing the names of statuses
print('Response_df size before merging with activity_df :', response_df.shape)
initi_cols = response_df.columns
response_df = pd.merge(response_df.drop(columns=['source_system']), activity_df.drop(columns=['source_system']), on = ['jasperUserId', 'sequenceId', 'historiesDb'], how = 'inner')

print('Response_df size before merging with activity_df:', response_df.shape)
print('New columns added from activity_df with activity_df:', set(response_df)^set(initi_cols))

Response_df size before merging with activity_df : (263674, 21)
Response_df size before merging with activity_df: (263674, 51)
New columns added from activity_df with activity_df: {'dateCreated', 'dateCompleted', 'omitted', 'product_name', 'templateId', 'product_code', 'sequenceStatus', 'alternate_timing', 'total_scored_items_time_elapsed', 'percentageNotPresented', 'sequenceName', 'notPresented', 'total_scored_items', 'total_items_remediated', 'total_scored_items_not_reached', 'program', 'total_scored_items_answered_correct', 'source_system', 'correct', 'percentageCorrect', 'activity_type', 'total_scored_items_answered', 'total_scored_items_omitted', 'percentageIncorrect', 'activity_title', 'unanswered', 'percentageOmitted', 'total_items_remediation_time', 'tutorMode', 'percentageUnanswered', 'incorrect', 'kbsEnrollmentId'}


In [43]:
#changing kbs_enrollment_id to string as in cleaning function to remove 0 kbseids we are comparing as string
response_df['kbsEnrollmentId'].fillna(value = 0, inplace = True)
response_df['kbsEnrollmentId'] = response_df['kbsEnrollmentId'].astype('str')

In [44]:
content_df.rename(columns = {'content_item_id' : 'contentItemId',
                            'content_item_name' : 'contentItemName',
                            'interaction_type_name' : 'interactiontypename',
                            'count_choices' : 'countchoices',
                            'parent_item_id' : 'parentid',
                            'parent_item_name' : 'parentname',
                            'correct_answer':'correctAnswer'}, inplace = True)

#category name vary based on problem, so check the data
#mergning with response_df
print('shape of response_df before merging with content_df :', response_df.shape)

response_df = pd.merge(response_df, content_df,
                       how = 'inner',
                      on = ['contentItemId', 'contentItemName'])

print('shape of response_df after merging with content_df :', response_df.shape)

shape of response_df before merging with content_df : (263674, 51)
shape of response_df after merging with content_df : (263674, 84)


In [45]:
#responses summary
response_summary = response_df[response_df['sequenceStatus'].str.lower() == 'completed'].groupby(['templateId', 'sequenceName', 'sectionName', 'jasperUserId'])[['sequenceId']].agg('count').reset_index()
response_summary.rename(columns = {'sequenceId' : 'num_responses'}, inplace = True)

response_summary = response_summary.groupby(['templateId', 'sequenceName'])[['num_responses']].agg(['max', 'min', 'median', 'count'])
response_summary.columns = ['max_resp', 'min_resp', 'median_resp', 'num_users']

response_summary

,,max_resp,min_resp,median_resp,num_users
templateId,sequenceName,,,,
12879,atomFredQbankShell,92,1,5.0,44598


In [46]:
test_map = pd.DataFrame()
test_map['templateId'] = [6039, 12879]
test_map['jasperSequenceName'] = ['comlexQbank',
                                 'atomFredQbankShell']

test_map['sectionName'] = [np.nan, np.nan]

test_map['numQues'] = [np.nan, np.nan]
test_map['responseThreshold'] = [np.nan, np.nan]
test_map['minutesAllowed'] = [np.nan, np.nan]

test_map

,templateId,jasperSequenceName,sectionName,numQues,responseThreshold,minutesAllowed
0,6039,comlexQbank,NaN,NaN,NaN,NaN
1,12879,atomFredQbankShell,NaN,NaN,NaN,NaN


# Connecting to Db for Repeats enrolls

In [17]:
engine = db_con()

query_frt = """select
    distinct ph.id kbsenrollmentid
from
    kbs_billing.purchase_history ph
    join bi_reporting.vw_product_detail prd on ph.product_id = prd.product_id
where
    lower(prd.product_subtype) in ('free trial');
    """

try:
    frt_enrols
except:
    frt_enrols = pd.read_sql(sql = query_frt, con = engine)

query_olc = """select
    distinct ph.id kbsenrollmentid
from
    kbs_billing.purchase_history ph
    join bi_reporting.vw_product_detail prd on ph.product_id = prd.product_id
where
    lower(prd.product_subtype) in ('online companion')
    ;"""

try:
    olc_enrols
except:
    olc_enrols = pd.read_sql(sql = query_olc, con = engine)

query_hsg_repeat = """select
    distinct ph.id kbsenrollmentid
from
    kbs_billing.purchase_history ph
    join bi_reporting.vw_product_detail prd on ph.product_id = prd.product_id
where
    ph.initial_delta_k_txn_code in ('404', '405', '406');
    --and ph.created_on >= '2020-01-01';
"""

try:
    repeaters
except:
    repeaters = pd.read_sql(sql = query_hsg_repeat, con = engine)
    repeaters['kbsenrollmentid'] = repeaters['kbsenrollmentid'].astype('str')

Enter db username : ········
Enter db password : ········
Enter database : redshiftapps


# Cleaning the data

In [47]:
#If we know the correct answers for all the items in response_df and correct answer column in response_df is not null
#then we can use no_correctAnswer = False
#and by default we work on contentItemId to get multiple correct answers
#we get correct answers df here, we need to filter items that are having multiple versions of items

cor_ans, dup_cor_ans = get_item_cor_ans(response_df, no_correctAnswer = True)

No duplicate correct answer


## Running cleaning function

In [48]:
data_pool = {'CI_old_version_dates' : pd.DataFrame(),
'CI_old_version_list' : pd.DataFrame(),
'CI_old_keys' : pd.DataFrame(),
'frt_enrols' : frt_enrols,
'olc_enrols' : olc_enrols,
'repeaters' : repeaters,
'section_map' : pd.DataFrame(),
'test_map' : test_map,
'seqHist_to_exclude' : pd.DataFrame(),
'cidf' : pd.DataFrame(),
'field_test_items' : pd.DataFrame(),
'ci_cols_to_include' : pd.DataFrame(),
'interaction_type_list' : pd.DataFrame()}

#field_test_items should have column name'contentItemName' to match with response_df

In [49]:
result, cleaning_info, rejects_df = clean_item_data(data_path = path,
                         results_path = results_path,
                        analysis_name = analysis_name,
                         resp = response_df,
                         remove_users_deleted_sequences = True,
                         remove_dup_CIs = True,
                         remove_no_kbsEID = True,
                         remove_deleted_sequences = True,
                         remove_impo_response_scored = True,
                         remove_impo_timing_seq = True,
                         remove_seq_w_tmq = True,
                         remove_staged_responses = False,
                         remove_FT_items = False,
                        data_pool = data_pool,
                         CI_remove_before_after = 'before', #applicable for old version dates
                         repeat_treatment = 'omit', #default omit
                         mSec_min_threshold = 5000, #there's y/n condition for timing if none provided
                         mSec_max_threshold = 600000, #so if condition is not needed.
                         sec_min_threshold = None,
                         sec_max_threshold = None,
                         remove_frt_users = True,
                         remove_olc_users = True,
                         remove_repeat_enrolls = True,
                         remove_tutor = True,
                         remove_ada_seq = True,
                         remove_untimed_seq = True,
                         remove_incomplete_seq = True,
                         seq_item_minutes_threshold = None, #input in terms of minutes here
                         seq_section_minutes_threshold = None, #input in terms of minutes here
                         seq_total_minutes_threshold = None, #input in terms of minutes here
                         qbank = False, #if qbank==false and section_map is not empty then sec_num_attempted is validated against given min_items_per_seq from secton_map
                         min_itmes_per_seq = None, #if qbank==true and section_map provided, sequence_num_attempted is validated against given min_items_per_seq
                         section_calc = True, #if section map provided but min_item_per_seq column not present in it & qbank == false then this gets activated for validating sec_perc_attempted vs section_resp_threshold(comes from section_map)
                         #test_map = test_map, #if section_map is empty this gets activated, and qbank==false then test_resp_threshold comes from test_map
                         #seq_item_resp_threshold = .75,
                         remove_unscored = False,
                         #seqHist_to_exclude = pd.DataFrame(), #should have column sequenceId in it
                         precombined_files = True, #used to add columns from cidf to response_df
                         #cidf = pd.DataFrame(), #pass cidf
                         #interaction_type_list = [], #pass interactionTypeIds in this list
                         #ci_cols_to_include = [], #pass columns names of cidf to include in response_df (ciname, ciid by default added if precombined files is True)
                         remove_repeat_test_administrations = False,
                        remove_seq_wo_dispseq = True)
    

Starting clean item data function at 2021-09-15 21:23:19
Total sequences at start:  47948
Total users at start:  3203
Unique items at start:  444
Total responses at start:  263674

Working on Disqualifiers :
User Removal
--------------------
Sequences with deleted names are removed:  1791
Users removed  50
Unique items removed  0
Responses removed  12717

Sequences with dupe content items within the same exam, removed: NONE

Sequences with no KBS EID removed: NONE

Sequence Removal
--------------------
Sequences with deleted names are removed: NONE

Sequences with bad records (response = 0 with score = 1), removed: NONE

Sequences with bad timing (mSecUsed < 0), removed:  6
Users removed  0
Unique items removed  0
Responses removed  49

Here are the new columns after joining all the test and section maps
{'test_num_ques', 'test_minutes_allowed', 'test_response_threshold'}

Sequences with too many questions in a section, removed: NONE

Item Removal
--------------------
Total item respon

## Exporting cleaning info & cleaned response_data

In [50]:
#Exporting cleaned response_data
result = result.sort_values(by=['studentId', 'activityId', 'displaySeq'], ignore_index= True)
result.to_csv(results_path+analysis_name+'_responseData.csv', index = False)

#Exporting cleaning info
cleaning_info.to_csv(results_path+analysis_name+'_cleaningInfo.csv')

#Exporting rejects info
rejects_df.sort_values(by=['studentId', 'templateId'], ignore_index = True).to_csv(results_path+analysis_name+'_rejects_info.csv', index = False)

# Creating User Level Matrices

In [51]:
#Reading cleaned data
all_resp = result.copy()

all_resp = all_resp.sort_values(by=['activityName', 'displaySeq'])

#col_name:filename
vars_for_matrix =  {'score':'Item_Scores',
                   'response':'Responses',
                   'mSecUsed':'Milliseconds_per_Item'}

#making matrices
big_matrix = make_user_level_matrices(all_resp,
                               vars_for_matrices = vars_for_matrix,
                               destination_file_path = results_path,
                               destination_file_name_prefix = '_User_level_',
                                analysis_name = analysis_name,
                                omit_code = '.',
                                not_seen_code = '-99',
                                use_display_order = True,
                               qbank = True)

Finished creating matrix for Item_Scores
Finished creating matrix for Responses
Finished creating matrix for Milliseconds_per_Item


# Metadata

## Sequence Level Information

In [53]:
activity_level_info = make_activity_level_info(df = result,
                                              results_path = results_path,
                                              analysis_name = analysis_name)

## User Level Information

In [54]:
user_info = make_user_level_info(df = result,
                                results_path = results_path,
                                analysis_name = analysis_name,
                                test_map = test_map)

## Item Level Information

In [55]:
item_level_info = make_item_level_info(df = result,
                                      content_df = content_df,
                                      results_path = results_path,
                                      analysis_name = analysis_name,
                                      corr_ans = cor_ans)